In [2]:
import snap
from os import path

In [3]:
BASE_PATH = "../data/academia.stackexchange.com"
FOLDED_USERID_GRAPH_PATH = path.join(BASE_PATH, "Userid_Folded_Graph.graph")
FOLDED_POSTID_GRAPH_PATH = path.join(BASE_PATH, "Postid_Folded_Graph.graph")
COMMUNITIES_PATH = path.join(BASE_PATH, 'communities.txt')
COMMUNITIES_VEC_PATH = path.join(BASE_PATH, 'communities.vector')

In [4]:
f_in = snap.TFIn(FOLDED_POSTID_GRAPH_PATH)
post_graph = snap.TUNGraph.Load(f_in)
print "nodes", post_graph.GetNodes()
print "edges", post_graph.GetEdges()

nodes 82014
edges -1974436891


In [ ]:
# CNM
assert snap.CntSelfEdges(post_graph) == 0

comm_vec = snap.TCnComV()
modularity = snap.CommunityCNM(post_graph, comm_vec)

f_out = snap.TFOut(COMMUNITIES_VEC_PATH)
comm_vec.Save(f_out)
f_out.Flush()

In [ ]:
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

print "communities", len(comm_vec)

with open(COMMUNITIES_PATH, 'w') as f:
    for i, comm in enumerate(comm_vec):
        f.write("#####Community {}#####\n".format(i))
        for node in comm:
            f.write(str(node) + '\n')
    f.write("The modularity of the network is {}".format(modularity))

In [ ]:
# modularity of communities
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

for i, comm in enumerate(comm_vec):
    community = snap.TIntV()
    for node in comm:
        community.Add(node)
    print 'Community {} modularity: {}'.format(i, snap.GetModularity(post_graph, community, post_graph.GetEdges()))